Mye av det samme som i modellportefølje_enkel_analyse.ipynb. Uten test av MLR forutsettningene. Lagrer viktige datapunkter i et dataframe gjennom en loop for å kunne kjøre mange kombinasjoner av ulike snitt

In [ ]:
import pandas as pd
import numpy as np
from numpy import nan
import yfinance as yf
import matplotlib.pyplot as plt
import pandas_datareader.data as reader
import datetime as dt
from sklearn import linear_model
import statsmodels.api as sm
import csv

tickersobx = pd.read_html('https://no.wikipedia.org/wiki/OSEBX-indeksen')[0]
tickersobx = tickersobx['Ticker'].to_list()
tickersobx = [i.replace('OSE: ','')for i in tickersobx]
tickersobx = [i + ('.OL')for i in tickersobx]
tickersobx = [i.replace('TIETOO.OL','TIETO.OL')for i in tickersobx]
tickers_to_remove = ['FJORD.OL', 'SRBANK.OL','NOFI.OL', 'AIRX.OL', 'AKH.OL', 'AGLX.OL']
tickersobx = [ticker for ticker in tickersobx if ticker not in tickers_to_remove]
historie = yf.download(tickersobx, '2015-01-01', '2022-01-01', interval='1d', threads = True)['Adj Close']
# Lager df med True variabler hvor kort sikt ligger over lang sikt

ShortStart = 25
LongStart = 150

numLong = 100
numShort = 50


# Create the DataFrame with the specified column and index names
alphaValues = pd.DataFrame(0, index=range(ShortStart, ShortStart+numShort), columns=range(LongStart, LongStart+numLong))
sharpeValues = pd.DataFrame(0, index=range(ShortStart, ShortStart+numShort), columns=range(LongStart, LongStart+numLong))
pValues = pd.DataFrame(0, index=range(ShortStart, ShortStart+numShort), columns=range(LongStart, LongStart+numLong))
stdValues = pd.DataFrame(0, index=range(ShortStart, ShortStart+numShort), columns=range(LongStart, LongStart+numLong))
diversifisertValues = pd.DataFrame(0, index=range(ShortStart, ShortStart+numShort), columns=range(LongStart, LongStart+numLong))
rValues = pd.DataFrame(0, index=range(ShortStart, ShortStart+numShort), columns=range(LongStart, LongStart+numLong))
adjustedrValues = pd.DataFrame(0, index=range(ShortStart, ShortStart+numShort), columns=range(LongStart, LongStart+numLong))
antHandlerValues = pd.DataFrame(0, index=range(ShortStart, ShortStart+numShort), columns=range(LongStart, LongStart+numLong))

for short in range(ShortStart, ShortStart+numShort):
    for long in range(LongStart, LongStart+numLong):

        SMALong = long
        SMAKort = short

        df_ma50 = historie.rolling(window=SMAKort).mean()
        df_ma200 = historie.rolling(window=SMALong).mean()
        df_bool = df_ma50 > df_ma200
        df_new = df_bool.mask(df_bool == False, np.nan)

        # Legger til alle kolonnenavnene med True verdi i innhold kolonne
        df_new['innhold'] = df_new.apply(lambda row: ','.join(row.index[row == True]), axis=1)

        #pct change
        returns = historie.pct_change()

        #Endrer til logreturns
        logreturns = np.log(1 + returns)


        # Legger til innholdet i porteføljen basert på signalene
        logreturns['innhold'] = df_new['innhold']

        # Lager en logreturns kolonne
        logreturns['summerteLogreturns'] = 0

        logreturnsUtenInnhold = logreturns.drop('innhold', axis=1)

        # Summerer logreturns av de som ligger inne i innholdkolonna
        prevInnhold = ''
        midlertidiglogreturn = 0
        antHandler = 0
        diversifisert = 100

        for index, row in logreturns.iterrows():

            innhold = logreturns.loc[index, 'innhold']
            innhold = innhold.split(',')
            midlertidiglogreturn = 0

            if (len(innhold)<diversifisert):
                diversifisert = len(innhold)

            for col_name, cell_value in row.iteritems():

                # Sjekker om den ligger i innhold, deretter legger til i midlertidiglogreturn
                if (col_name in innhold):
                    try:
                        midlertidiglogreturn += cell_value/(len(innhold)-1)
                    except ZeroDivisionError:
                        print('')

                # Kjøp
                if ((col_name in innhold) and (col_name not in prevInnhold)):
                    antHandler += 1

                # Selg
                if ((col_name not in innhold) and (col_name in prevInnhold)):
                    antHandler += 1

            # Legger til midlertidiglogreturns i summerteLogreturns
            logreturnsUtenInnhold.loc[index, 'summerteLogreturns'] += midlertidiglogreturn
            prevInnhold = innhold

        # Kummulerer logreturns, gjør de om til aritmetisk return
        logreturnsUtenInnhold['summerteLogreturns'] = np.exp(np.log(1 + logreturnsUtenInnhold['summerteLogreturns']).cumsum())


        # Dropper timer, minutter og sekunder fra index
        logreturnsUtenInnhold.index = logreturnsUtenInnhold.index.strftime('%Y-%m-%d')

        # Fra odegardsjekk.ipynb, legger inn odegaard-verdier i en df og endrer index til datetime-format
        df = pd.read_csv('./dataset/pricing_factors_daily.csv', index_col='date')
        df.index = pd.to_datetime(df.index, format='%Y%m%d', errors='coerce')

        portfolioDF = pd.DataFrame(columns=['portfolio'], index=['date'])

        rows_to_concat = []
        for index, row in logreturnsUtenInnhold.iterrows():
            if index in df.index:
                indeks = index
                new_row = {indeks: logreturnsUtenInnhold.loc[index,'summerteLogreturns']}
                rows_to_concat.append(pd.DataFrame.from_dict(new_row, orient='index', columns=['portfolio']))

        if rows_to_concat:
            portfolioDF = pd.concat([portfolioDF] + rows_to_concat)

        portfolioDF.index.name = 'date'
        portfolioDF = portfolioDF.drop('date')
        portfolioDF = portfolioDF.pct_change()
        portfolioDF = portfolioDF[SMALong:]

        df = pd.read_csv('./dataset/pricing_factors_daily.csv', index_col='date')

        df = df[['SMB', 'HML']]
        rf = pd.read_csv('./dataset/rf_daily.csv', index_col='date')

        new_df = df.copy()
        new_df['rf'] = 0

        # Convert the index to datetime objects
        df.index = pd.to_datetime(df.index, format='%Y%m%d')
        rf.index = pd.to_datetime(rf.index, format='%Y-%m-%d')

        # Merge the two DataFrames based on the index
        df_mergedPricingFactors = pd.merge(df, rf, left_index=True, right_index=True, how='inner')
        merged_df = pd.merge(left=df_mergedPricingFactors, right=portfolioDF, left_index=True, right_index=True, how='inner')
        df_mergedPricingFactors.index = pd.to_datetime(df_mergedPricingFactors.index, format='%Y%m%d')

        for index, row in portfolioDF.iterrows():
            if index in df_mergedPricingFactors.index:
                new_row = {'SMB': df_mergedPricingFactors.loc[indeks, 'SMB'], 'HML': df_mergedPricingFactors.loc[indeks, 'HML'], 'rf(1d)':df_mergedPricingFactors.loc[indeks, 'rf(1d)'], 'portfolio':portfolioDF.loc[indeks, 'portfolio']}
                new_df = pd.DataFrame(new_row, index=[index], columns=['SMB', 'HML', 'rf(1d)', 'portfolio'])
                merged_df = pd.concat([merged_df, new_df])

        mr = pd.read_csv('./dataset/market_portfolios_daily.csv', index_col='date')
        mr = mr.drop('date')
        mr = mr['OSEAX']
        mr.index = pd.to_datetime(mr.index, format='%Y%m%d')
        merged_df = pd.merge(left=merged_df, right=mr, left_index=True, right_index=True, how='inner')
        try:
            portfolioDF = portfolioDF.drop('2017-04-13')
        except KeyError:
            print("")

        for index, row in portfolioDF.iterrows():
            if index in mr.index:
                new_row = {'OSEAX' : mr.loc[index], 'SMB': df_mergedPricingFactors.loc[index, 'SMB'], 'HML': df_mergedPricingFactors.loc[index, 'HML'], 'rf(1d)':df_mergedPricingFactors.loc[index, 'rf(1d)'], 'portfolio':portfolioDF.loc[index, 'portfolio']}
                new_df = pd.DataFrame(new_row, index=[index], columns=['SMB', 'HML', 'rf(1d)', 'portfolio', 'OSEAX'])
                merged_df = pd.concat([merged_df, new_df])
        merged_df = merged_df.rename(columns={'rf(1d)': 'rf'})

        merged_df['OSEAX'] = merged_df['OSEAX'].astype(float)
        merged_df['portfolio-rf'] = merged_df.portfolio - merged_df.rf
        merged_df['MRKT-rf'] = merged_df.OSEAX - merged_df.rf

        Y = merged_df['portfolio-rf']
        X = merged_df[['MRKT-rf', 'SMB', 'HML']]

        X_sm = sm.add_constant(X)

        model = sm.OLS(Y, X_sm)
        results = model.fit()

        coef = results.params
        tvals = results.tvalues
        pvals = results.pvalues
        results_df = pd.DataFrame({'coef': coef, 'tvals': tvals, 'pvals': pvals})


        merged_df = np.log(1 + merged_df)
        merged_df = np.exp(np.log(1 + merged_df).cumsum())

        std = merged_df['portfolio'].std()

        ind = merged_df.index[-1]
        sharpe = (merged_df.loc[ind, 'portfolio']-merged_df.loc[ind, 'rf'])/std

        print(f'Sharpe ratio of portfolio: {sharpe}')
        
        print('Alpha')
        print(results.params['const'])

        print('P-value')
        print(float(results.pvalues['const']))

        alphaValues.at[short, long] = float(results.params['const'])
        pValues.at[short, long] = float(results.pvalues['const'])
        sharpeValues.at[short, long] = float(sharpe)
        stdValues.at[short, long] = float(std)
        diversifisertValues.at[short, long] = float(diversifisert)
        rValues.at[short, long] = float(results.rsquared)
        adjustedrValues.at[short, long] = float(results.rsquared_adj)
        antHandlerValues.at[short, long] = float(antHandler)
print(alphaValues)

Lagring av resultat til csv-fil

In [ ]:
alphaValues.to_csv('alphaValues.1015.csv', index=True)